Create a virtual environment:


In [41]:
!python -m venv rag_env
!source rag_env/bin/activate

The virtual environment was not created successfully because ensurepip is not
available.  On Debian/Ubuntu systems, you need to install the python3-venv
package using the following command.

    apt install python3.10-venv

You may need to use sudo with that command.  After installing the python3-venv
package, recreate your virtual environment.

Failing command: /content/rag_env/bin/python3

/bin/bash: line 1: rag_env/bin/activate: No such file or directory


In [42]:
pip install pandas numpy streamlit sentence-transformers langchain milvus pymilvus openai PyPDF2 SpeechRecognition ffmpeg


Create a RAG pipeline


Insstalling Dependencies


In [43]:
# Install necessary libraries
!pip install sentence-transformers
!pip install transformers
!pip install pymilvus
!pip install torch
!pip install docker
!pip install streamlit


Uplaoding the relavant documents


In [44]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


data cleaning for text file

In [45]:
import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab') # Download the missing 'punkt_tab' data package

# Load stop words
stop_words = set(stopwords.words('english'))

# Function to clean the text: Remove punctuation, stop words, and lowercasing
def clean_text(text):
    # Convert to lowercase
    text = text.lower()

    # Remove punctuation
    text = ''.join([char for char in text if char not in string.punctuation])

    # Tokenize the text
    words = word_tokenize(text)

    # Remove stop words
    cleaned_words = [word for word in words if word not in stop_words]

    # Rejoin words to form cleaned text
    cleaned_text = ' '.join(cleaned_words)

    return cleaned_text

# Path to the text file
text_file_path = '/content/drive/MyDrive/RAG_text.txt'

# Read the text file
with open(text_file_path, 'r') as file:
    text_data = file.read()

# Clean the text
cleaned_text = clean_text(text_data)

# Print cleaned text
print("Cleaned Text from txt file:\n")
print(cleaned_text[:500])  # Print the first 500 characters

Cleaned Text from txt file:

sky blue


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Tokenize the Text (Optional but Recommended for Embeddings)
Tokenizing helps break the cleaned text into smaller chunks (words or sub-words), which are often required for embedding models.


In [46]:
# Tokenize the cleaned text
tokens = word_tokenize(cleaned_text)

# Print the first few tokens
print("Tokens from the cleaned text:\n")
print(tokens[:50])  # Print first 50 tokens


Tokens from the cleaned text:

['sky', 'blue']


Prepare for Embedding Generation
Once the text is cleaned and tokenized, it’s ready for generating embeddings using a model like Sentence-Transformer. We can now prepare this data for that step

In [47]:
# Prepare the cleaned and tokenized text for embeddings
# You can save these tokens or cleaned text to a variable, or store them for future use
processed_data = cleaned_text  # or tokens if you need them in tokenized form

# Optionally, you can store it in a list or other data structure if needed
documents = [processed_data]  # This list will contain the processed text

# Print the final processed text (cleaned and tokenized)
print("Final Processed Data (cleaned text):\n")
print(processed_data[:500])  # Print first 500 characters


Final Processed Data (cleaned text):

sky blue


 Save Processed Data for Future Use (Optional)
You may want to save this cleaned and processed data for future steps (like embedding generation or storing in the vector database).

In [50]:
# Save cleaned text to a new file (if required)
output_file_path = '/content/drive/MyDrive/MES_Files/RAG_processed_text.txt'
with open(output_file_path, 'w') as output_file:
    output_file.write(processed_data)

print(f"Processed text has been saved to {output_file_path}")


Processed text has been saved to /content/drive/MyDrive/MES_Files/RAG_processed_text.txt


**PDF FORMAT**

In [51]:
!pip install pymupdf
!pip install nltk


In [52]:
import fitz  # PyMuPDF
import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')

# Load stop words
stop_words = set(stopwords.words('english'))


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Extract Text from PDF

In [53]:
# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

# Path to the PDF file
pdf_file_path = '/content/drive/MyDrive/RAG_text_pdf.pdf'

# Extract text from the PDF
pdf_text = extract_text_from_pdf(pdf_file_path)

# Print the first 500 characters of the extracted text
print("Extracted Text from PDF file:\n")
print(pdf_text[:500])  # Print first 500 characters to preview


Extracted Text from PDF file:

The sky is blue



Clean the Text


In [55]:
# Function to clean the text: Remove punctuation, stop words, and lowercasing
def clean_text(text):
    # Convert to lowercase
    text = text.lower()

    # Remove punctuation
    text = ''.join([char for char in text if char not in string.punctuation])

    # Tokenize the text
    words = word_tokenize(text)

    # Remove stop words
    cleaned_words = [word for word in words if word not in stop_words]

    # Rejoin words to form cleaned text
    cleaned_text = ' '.join(cleaned_words)

    return cleaned_text

# Clean the extracted PDF text
cleaned_pdf_text = clean_text(pdf_text)

# Print cleaned text
print("Cleaned Text from PDF file:\n")
print(cleaned_pdf_text[:500])  # Print first 500 characters of cleaned text


Cleaned Text from PDF file:

sky blue


Tokenize the Cleaned Text

In [56]:
# Tokenize the cleaned text
tokens = word_tokenize(cleaned_pdf_text)

# Print the first few tokens
print("Tokens from the cleaned PDF text:\n")
print(tokens[:50])  # Print first 50 tokens


Tokens from the cleaned PDF text:

['sky', 'blue']


Prepare for Further Use (Embedding Generation)
Finally, you can store the cleaned and tokenized text, or the raw cleaned text, for future use like embedding generation.

In [57]:
# Store the processed data (cleaned text) for future use
processed_pdf_data = cleaned_pdf_text  # Or tokens if needed

# Print the processed data
print("Processed Data (Cleaned Text):\n")
print(processed_pdf_data[:500])  # Print first 500 characters of cleaned data


Processed Data (Cleaned Text):

sky blue


Optionally Save the Cleaned PDF Text
You might want to save the cleaned PDF text to a file for later use.

python
Copy code


In [58]:
import os

# Save cleaned text to a new file (optional)
output_pdf_path = '/content/drive/MyDrive/MES_Files/processed_pdf_text.txt'

# Create the directory if it doesn't exist
os.makedirs(os.path.dirname(output_pdf_path), exist_ok=True)

with open(output_pdf_path, 'w') as output_file:
    output_file.write(processed_pdf_data)

print(f"Processed PDF text has been saved to {output_pdf_path}")

Processed PDF text has been saved to /content/drive/MyDrive/MES_Files/processed_pdf_text.txt


**AUDIO FORMAT**

In [59]:
!apt-get install ffmpeg
!pip install SpeechRecognition pydub
!pip install nltk


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


Extract Audio from Video
We first extract the audio from the video file since transcription tools work on audio. Use FFmpeg for this.

In [60]:
import os

# Paths
video_file_path = '/content/drive/MyDrive/RAG_video.mp4'  # Input video file
audio_file_path = '/content/drive/MyDrive/MES_Files/audio_from_video.wav'  # Extracted audio file

# Extract audio from video
os.system(f"ffmpeg -i {video_file_path} -q:a 0 -map a {audio_file_path}")
print(f"Audio extracted and saved at {audio_file_path}")


Audio extracted and saved at /content/drive/MyDrive/MES_Files/audio_from_video.wav


Enhance Audio Quality
Enhance the audio to improve transcription accuracy. This involves noise reduction, normalization, and frequency filtering.

Code for Noise Reduction:

In [61]:
!pip uninstall noisereduce
!pip uninstall ffmpeg-python
!pip install noisereduce
!pip install ffmpeg-python


Found existing installation: noisereduce 3.0.3
Uninstalling noisereduce-3.0.3:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/noisereduce-3.0.3.dist-info/*
    /usr/local/lib/python3.10/dist-packages/noisereduce/*
Proceed (Y/n)? y
  Successfully uninstalled noisereduce-3.0.3
Found existing installation: ffmpeg-python 0.2.0
Uninstalling ffmpeg-python-0.2.0:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/ffmpeg/*
    /usr/local/lib/python3.10/dist-packages/ffmpeg_python-0.2.0.dist-info/*
  Would not remove (might be manually added):
    /usr/local/lib/python3.10/dist-packages/ffmpeg/audio.py
    /usr/local/lib/python3.10/dist-packages/ffmpeg/image.py
    /usr/local/lib/python3.10/dist-packages/ffmpeg/stream.py
    /usr/local/lib/python3.10/dist-packages/ffmpeg/video.py
Proceed (Y/n)? y
  Successfully uninstalled ffmpeg-python-0.2.0
  Using cached noisereduce-3.0.3-py3-none-any.whl.metadata (14 kB)
Using cached noisereduce-3.0.3-py3-none-any.whl (22 kB)


  Using cached ffmpeg_python-0.2.0-py3-none-any.whl.metadata (1.7 kB)
Using cached ffmpeg_python-0.2.0-py3-none-any.whl (25 kB)


In [63]:
# Install required libraries
!pip install ffmpeg-python noisereduce pydub

# Import necessary libraries
import os
import ffmpeg
from pydub import AudioSegment
from noisereduce import reduce_noise
import numpy as np
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Path to the video file in Google Drive
video_file_path = '/content/drive/My Drive/RAG_assignment/RAG_video.mp4'  # Replace with your file path

# Extract audio from the video
output_audio_path = '/content/extracted_audio.wav'
(
    ffmpeg
    .input(video_file_path)
    .output(output_audio_path, format='wav')
    .run(overwrite_output=True)
)
print(f"Audio extracted to: {output_audio_path}")

# Load the extracted audio
audio = AudioSegment.from_file(output_audio_path)

# Convert audio to numpy array
samples = np.array(audio.get_array_of_samples())

# Perform noise reduction
# Here, we assume the first 1 second is noise for profile creation
sample_rate = audio.frame_rate
noise_sample = samples[:sample_rate]  # Use the first second as noise profile
reduced_noise = reduce_noise(y=samples, sr=sample_rate, y_noise=noise_sample)

# Save the noise-reduced audio
os.makedirs('/content/drive/My Drive/MES_files/redused_audio.wav', exist_ok=True)
reduced_audio_path = '/content/drive/My Drive/MES_Files/redused_audio.wav'
reduced_audio = AudioSegment(
    reduced_noise.tobytes(),
    frame_rate=sample_rate,
    sample_width=audio.sample_width,
    channels=audio.channels
)
reduced_audio.export(reduced_audio_path, format='wav')
print(f"Noise-reduced audio saved to: {reduced_audio_path}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Audio extracted to: /content/extracted_audio.wav
Noise-reduced audio saved to: /content/drive/My Drive/MES_Files/redused_audio.wav


(Optional) Split Audio into Smaller Chunks
If the audio is long, split it into smaller chunks for better transcription results.

In [64]:
from pydub import AudioSegment

# Directory for audio chunks
chunk_dir = '/content/drive/MyDrive/MES_Files/audio_chunks/'
os.makedirs(chunk_dir, exist_ok=True)

# Load the enhanced audio
audio = AudioSegment.from_wav("/content/drive/My Drive/RAG_assignment/redused_audio.wav")

# Split into 30-second chunks
chunk_length_ms = 30 * 1000  # 30 seconds in milliseconds
chunks = [audio[i:i + chunk_length_ms] for i in range(0, len(audio), chunk_length_ms)]

# Save each chunk
chunk_paths = []
for idx, chunk in enumerate(chunks):
    chunk_path = os.path.join(chunk_dir, f'chunk_{idx}.wav')
    chunk.export(chunk_path, format="wav")
    chunk_paths.append(chunk_path)
    print(f"Saved chunk {idx} to {chunk_path}")

print(f"Audio has been split into {len(chunk_paths)} chunks.")


Saved chunk 0 to /content/drive/MyDrive/MES_Files/audio_chunks/chunk_0.wav
Saved chunk 1 to /content/drive/MyDrive/MES_Files/audio_chunks/chunk_1.wav
Saved chunk 2 to /content/drive/MyDrive/MES_Files/audio_chunks/chunk_2.wav
Saved chunk 3 to /content/drive/MyDrive/MES_Files/audio_chunks/chunk_3.wav
Saved chunk 4 to /content/drive/MyDrive/MES_Files/audio_chunks/chunk_4.wav
Saved chunk 5 to /content/drive/MyDrive/MES_Files/audio_chunks/chunk_5.wav
Saved chunk 6 to /content/drive/MyDrive/MES_Files/audio_chunks/chunk_6.wav
Saved chunk 7 to /content/drive/MyDrive/MES_Files/audio_chunks/chunk_7.wav
Saved chunk 8 to /content/drive/MyDrive/MES_Files/audio_chunks/chunk_8.wav
Saved chunk 9 to /content/drive/MyDrive/MES_Files/audio_chunks/chunk_9.wav
Saved chunk 10 to /content/drive/MyDrive/MES_Files/audio_chunks/chunk_10.wav
Audio has been split into 11 chunks.


Transcribe Audio to Text
Transcribe the enhanced audio or its chunks using Whisper for robust transcription.

Install Whisper

In [65]:
!pip install openai-whisper ffmpeg


Transcription code

In [66]:
import whisper

# Load the Whisper model
model = whisper.load_model("base")

# Transcribe the enhanced audio
result = model.transcribe(reduced_audio_path)
transcribed_text = result["text"]
print("Transcribed Text:\n", transcribed_text)


  checkpoint = torch.load(fp, map_location=device)

  warnings.warn("FP16 is not supported on CPU; using FP32 instead")



Transcribed Text:
  What is MES? MES stands for Manufacturing Execution System, meaning a control system for monitoring and managing work and process on factory floor. But that's an oversimplification of what a successful MES software implementation can do for manufacturers. MES provides detailed resource scheduling and status, production, dispatching, sequencing, traceability, genealogy, inventory, quality assurance, and maintenance management, document control, performance, analysis, and work. MES is crucial for manufacturers because it exists in a space between business-oriented applications, like ERP and SCADA HMI systems designed to directly control plant or operations. While an ERP can help allocate resources, it lacks the level of detail that MES provides. MES allows real-time, minute to minute, or quicker resource scheduling, as well as handling execution and dispatch. MES connects production to executive decision making by controlling, tracking, and documenting the transformat

For audio chunks

In [67]:
# Transcribe each chunk
transcribed_chunks = []
for chunk_path in chunk_paths:
    result = model.transcribe(chunk_path)
    transcribed_chunks.append(result["text"])

# Combine transcribed text
final_transcribed_text = " ".join(transcribed_chunks)
print("Final Transcribed Text:\n", final_transcribed_text)


  warnings.warn("FP16 is not supported on CPU; using FP32 instead")

  warnings.warn("FP16 is not supported on CPU; using FP32 instead")

  warnings.warn("FP16 is not supported on CPU; using FP32 instead")

  warnings.warn("FP16 is not supported on CPU; using FP32 instead")

  warnings.warn("FP16 is not supported on CPU; using FP32 instead")

  warnings.warn("FP16 is not supported on CPU; using FP32 instead")

  warnings.warn("FP16 is not supported on CPU; using FP32 instead")

  warnings.warn("FP16 is not supported on CPU; using FP32 instead")

  warnings.warn("FP16 is not supported on CPU; using FP32 instead")

  warnings.warn("FP16 is not supported on CPU; using FP32 instead")

  warnings.warn("FP16 is not supported on CPU; using FP32 instead")



Final Transcribed Text:
  What is MES? MES stands for Manufacturing Execution System, meaning a control system for monitoring and managing work and process on factory floor. But that's an oversimplification of what a successful MES software implementation can do for manufacturers. MES provides detailed resource scheduling and status, production dispassion sequences.  Traceability, Genealogy, Inventory, Quality Assurance, and Maintenance Management, Document Control, Performance, Analysis, and more. MES is crucial for manufacturers because it exists in a space between business-oriented applications like ERP and SCADA HMI systems designed to directly control plant or operations. All in ERP can help allocate resources. It lacks the level of detail that MES provides.  MES allows for real-time, minute to minute, or quicker, resource scheduling, as well as handling execution and dispatch. MES connects production to executive decision making by controlling, tracking, and documenting the trans

Clean the Transcribed Text
Remove unnecessary noise like punctuation, stopwords, and convert to lowercase.

In [68]:
import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')

# Stopwords and cleaning function
stop_words = set(stopwords.words('english'))

def clean_text(text):
    text = text.lower()  # Convert to lowercase
    text = ''.join([char for char in text if char not in string.punctuation])  # Remove punctuation
    words = word_tokenize(text)  # Tokenize
    words = [word for word in words if word not in stop_words]  # Remove stopwords
    return ' '.join(words)

# Clean the transcribed text
cleaned_text = clean_text(transcribed_text if len(chunk_paths) == 0 else final_transcribed_text)
print("Cleaned Transcribed Text:\n", cleaned_text)


Cleaned Transcribed Text:
 mes mes stands manufacturing execution system meaning control system monitoring managing work process factory floor thats oversimplification successful mes software implementation manufacturers mes provides detailed resource scheduling status production dispassion sequences traceability genealogy inventory quality assurance maintenance management document control performance analysis mes crucial manufacturers exists space businessoriented applications like erp scada hmi systems designed directly control plant operations erp help allocate resources lacks level detail mes provides mes allows realtime minute minute quicker resource scheduling well handling execution dispatch mes connects production executive decision making controlling tracking documenting transformation raw materials finished goods realtime information gives current accurate view plantfloor operations especially combined dashboards display current status rates compared historical data well sche

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Saving the Transcribed Text
If you want to save the transcribed or cleaned text into a file for later use, use the following code:

Save the Raw Transcription:

In [69]:
raw_text_path = '/content/drive/MyDrive/MES_Files/transcribed_text.txt'
with open(raw_text_path, 'w') as f:
    f.write(transcribed_text if len(chunk_paths) == 0 else final_transcribed_text)
print(f"Transcribed text saved at {raw_text_path}")


Transcribed text saved at /content/drive/MyDrive/MES_Files/transcribed_text.txt


Save the Cleaned Transcription:
python
Copy code


In [70]:
cleaned_text_path = '/content/drive/MyDrive/MES_Files/cleaned_transcribed_text.txt'
with open(cleaned_text_path, 'w') as f:
    f.write(cleaned_text)
print(f"Cleaned transcribed text saved at {cleaned_text_path}")


Cleaned transcribed text saved at /content/drive/MyDrive/MES_Files/cleaned_transcribed_text.txt


MAKING EMBEDDINGS OUT OF THEM

In [71]:
!pip install sentence-transformers


In [78]:
from sentence_transformers import SentenceTransformer
import numpy as np

# Load the SentenceTransformer model
model = SentenceTransformer('all-MiniLM-L6-v2')  # Efficient model for embeddings

# Define paths for your cleaned text files
# Define paths for your cleaned text files (check and update the paths if necessary)
cleaned_text_files = {
    'txt': '/content/drive/MyDrive/MES_Files/processed_output.txt',  # Cleaned text from TXT file
    'pdf': '/content/drive/MyDrive/MES_Files/processed_pdf_text.txt', # Cleaned text from PDF
    'audio': '/content/drive/MyDrive/MES_Files/cleaned_transcribed_text.txt'  # Cleaned text from Transcription
}



# Dictionary to hold the embeddings
embeddings_dict = {}

# Load and encode the text files
for key, file_path in cleaned_text_files.items():
    try:
        with open(file_path, 'r') as f:
            cleaned_text = f.read()

        # Generate embeddings for each cleaned text
        embeddings = model.encode([cleaned_text])

        # Store embeddings in the dictionary
        embeddings_dict[key] = embeddings

        # Save embeddings for each format separately
        embeddings_file_path = f"/content/drive/MyDrive/MES_Files/embeddings_{key}.npy"
        np.save(embeddings_file_path, embeddings)
        print(f"Embeddings for {key} saved at {embeddings_file_path}")

    except FileNotFoundError:
        print(f"Error: File not found at {file_path}. Please check the file path and ensure the file exists.")

# Optionally: Print out the shape of embeddings to confirm
for key, emb in embeddings_dict.items():
    print(f"Embeddings for {key} shape: {np.array(emb).shape}")

Embeddings for txt saved at /content/drive/MyDrive/MES_Files/embeddings_txt.npy
Embeddings for pdf saved at /content/drive/MyDrive/MES_Files/embeddings_pdf.npy
Embeddings for audio saved at /content/drive/MyDrive/MES_Files/embeddings_audio.npy
Embeddings for txt shape: (1, 384)
Embeddings for pdf shape: (1, 384)
Embeddings for audio shape: (1, 384)


 FAISS VECTOR DATABASE

In [79]:
!pip install faiss-cpu  # For CPU-only version
# or
!pip install faiss-gpu  # For GPU-accelerated version (if you have a compatible GPU)


In [80]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Your embeddings will likely be located at '/content/drive/My Drive/...'
# Example path to your files on Google Drive
path_to_txt_embeddings = '/content/drive/MyDrive/MES_Files/embeddings_txt.npy'
path_to_pdf_embeddings = '/content/drive/MyDrive/MES_Files/embeddings_pdf.npy'
path_to_audio_embeddings = '/content/drive/MyDrive/MES_Files/embeddings_audio.npy'


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [82]:
import numpy as np

# Load the embeddings from Google Drive
embeddings_txt = np.load(path_to_txt_embeddings)
embeddings_pdf = np.load(path_to_pdf_embeddings)
embeddings_audio = np.load(path_to_audio_embeddings)

# Check their shapes to make sure they are compatible
print("Text Embedding Shape:", embeddings_txt.shape)
print("PDF Embedding Shape:", embeddings_pdf.shape)
print("Audio Embedding Shape:", embeddings_audio.shape)


Text Embedding Shape: (1, 384)
PDF Embedding Shape: (1, 384)
Audio Embedding Shape: (1, 384)


In [83]:
import faiss

# Check the dimensionality of the embeddings (assuming all have the same dimension)
embedding_dimension = embeddings_txt.shape[1]  # Example: 384

# Create the FAISS index (using IndexFlatIP for cosine similarity)
index = faiss.IndexFlatIP(embedding_dimension)

# Combine all the embeddings into a single numpy array (stacking vertically)
all_embeddings = np.vstack((embeddings_txt, embeddings_pdf, embeddings_audio))

# Add all the embeddings to the FAISS index
index.add(all_embeddings)

# Verify the number of vectors in the index
print("Number of vectors in the FAISS index:", index.ntotal)


Number of vectors in the FAISS index: 3


In [84]:
# Example query: create a random vector to simulate a query (this should ideally be an actual query embedding)
query_embedding = np.random.random((1, embedding_dimension)).astype('float32')

# Retrieve the top 5 nearest neighbors
k = 5
distances, indices = index.search(query_embedding, k)

print("Top 5 nearest neighbors' indices:", indices)
print("Top 5 nearest neighbors' distances:", distances)


Top 5 nearest neighbors' indices: [[ 1  0  2 -1 -1]]
Top 5 nearest neighbors' distances: [[ 1.9852403e-01  1.9852403e-01 -5.7216984e-01 -3.4028235e+38
  -3.4028235e+38]]


In [85]:
# Save the FAISS index to a file
faiss.write_index(index, '/content/drive/My Drive/faiss_index.index')

# You can later read the index back from the saved file
# index = faiss.read_index('/content/drive/My Drive/faiss_index.index')


In [86]:
# Example query: Create a random query vector for testing (in real scenarios, this will be your query embedding)
query_embedding = np.random.random((1, embedding_dimension)).astype('float32')

# Retrieve the top 5 nearest neighbors
k = 5
distances, indices = index.search(query_embedding, k)

# Display the results
print("Top 5 nearest neighbors' indices:", indices)
print("Top 5 nearest neighbors' distances:", distances)


Top 5 nearest neighbors' indices: [[ 1  0  2 -1 -1]]
Top 5 nearest neighbors' distances: [[ 3.5174194e-01  3.5174194e-01 -7.1926439e-01 -3.4028235e+38
  -3.4028235e+38]]


**Prepare Query Embedding**

Step 1: Load the necessary libraries and the embeddings
python
Copy code


In [87]:
!pip install SpeechRecognition


In [88]:
!pip install pydub
!pip install SpeechRecognition
!apt-get install ffmpeg  # Install ffmpeg if you are using MP3 files


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [90]:
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
import pickle
import os

# Step 1: Load the Sentence Transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Step 2: Load your embeddings from Google Drive (make sure they are in numpy format)
embeddings_txt = np.load('/content/drive/MyDrive/MES_Files/embeddings_txt.npy')
embeddings_pdf = np.load('/content/drive/MyDrive/MES_Files/embeddings_pdf.npy')
embeddings_audio = np.load('/content/drive/MyDrive/MES_Files/embeddings_audio.npy')

# Step 3: Load the original documents (actual text files or other formats)
documents_txt = ["RAG_text"

]  # Example for text documents

documents_pdf = ["RAG_text_pdf"

]  # Example for PDF documents

documents_audio = ["RAG_video"

]  # Example for audio documents

# Step 4: Combine all embeddings and documents
all_embeddings = np.concatenate([embeddings_txt, embeddings_pdf, embeddings_audio], axis=0)
all_documents = documents_txt + documents_pdf + documents_audio  # Combine all document contents

# Step 5: Create or load FAISS index
dim = all_embeddings.shape[1]  # Dimensionality of the embeddings
index = faiss.IndexFlatL2(dim)  # Using L2 distance metric
index.add(all_embeddings)  # Add embeddings to the index

# Step 6: Map FAISS indices to documents
document_map = {i: all_documents[i] for i in range(len(all_documents))}

# Step 7: Save the document_map for later retrieval (in case you want to load it later)
with open('/content/drive/MyDrive/document_map.pkl', 'wb') as f:
    pickle.dump(document_map, f)

# Step 8: Function to retrieve corresponding document based on FAISS search result
def retrieve_documents_from_faiss(query_embedding, index, document_map, k=5):
    # Perform the search
    _, indices = index.search(query_embedding, k)  # Get top k results

    # Retrieve the documents based on the FAISS indices, ignoring -1 indices
    retrieved_documents = [document_map[i] for i in indices[0] if i != -1]
    return retrieved_documents
# step 9
file_path = "/content/drive/MyDrive/MES_Files/cleaned_transcribed_text.txt"  # Update with the correct path
if not os.path.exists(file_path):
    raise FileNotFoundError(f"The file '{file_path}' does not exist. "
                            f"Please make sure the file is present or provide the correct path.")

with open(file_path, "r") as file:
    transcribed_text = file.read()

# Step 10: Generate the embedding for the transcribed text
query_embedding = model.encode([transcribed_text])

# Step 11: Retrieve relevant documents based on the query embedding
retrieved_docs = retrieve_documents_from_faiss(query_embedding, index, document_map)

# Step 12: Print the retrieved documents
for doc in retrieved_docs:
    print(doc)

RAG_video
RAG_text
RAG_text_pdf


Step 2: Write a function to process the query and search the database

In [91]:
def search_query(query, index, model, top_k=5):
    # Convert query to embedding
    query_embedding = model.encode([query])

    # Perform the search in the FAISS index
    distances, indices = index.search(query_embedding, top_k)

    return distances, indices


Step 3: Retrieve the most relevant results from FAISS

In [92]:
# Example query related to MES
query = "What is the process of material management in MES?"

# Search the query in the FAISS index
distances, indices = search_query(query, index, model)

# Print the most relevant results (top_k closest documents)
print("Top results for query:", query)
for i, idx in enumerate(indices[0]):
    print(f"Result {i+1}:")
    if idx < len(embeddings_txt):
        source = 'Text (TXT)'
    elif idx < len(embeddings_txt) + len(embeddings_pdf):
        source = 'Text (PDF)'
    else:
        source = 'Text (Audio)'

    print(f"Source: {source} | Distance: {distances[0][i]}")
    # You can return the actual document or text associated with the embedding (you would need to store the documents in a list)
    print(f"Document Content: {all_embeddings[idx]} \n\n")


Top results for query: What is the process of material management in MES?
Result 1:
Source: Text (Audio) | Distance: 0.7817662954330444
Document Content: [-2.81377193e-02  1.08364159e-02 -1.68290976e-02  8.96473229e-03
 -1.91974621e-02 -5.60550019e-02  5.99054396e-02  1.10962495e-01
 -5.02063185e-02  5.70579022e-02 -2.01693010e-02 -6.16779067e-02
 -2.13465709e-02  6.18569599e-03 -4.46746573e-02  1.49200056e-02
  1.22437462e-01 -5.58001511e-02 -5.03663011e-02 -2.17202753e-02
  7.73597835e-03 -3.19604501e-02 -6.65669590e-02  2.67106108e-03
 -1.31063893e-01  2.59651872e-03 -8.40959772e-02  2.72645261e-02
  3.65814776e-04 -1.05566032e-01 -4.58217040e-02  3.08121573e-02
  1.31280765e-01  3.19057405e-02  5.69830835e-03  5.82956187e-02
  7.89410099e-02 -8.47371016e-03 -9.36729833e-03 -7.55671933e-02
 -9.70806852e-02 -4.53101881e-02 -3.26452293e-02  2.01059170e-02
  2.81315763e-02  3.52940038e-02 -6.04527779e-02 -1.22288465e-02
  4.56640571e-02  1.59336962e-02 -6.68828934e-02 -1.75219271e-02
 

In [93]:
import faiss
import numpy as np

# Assume 'faiss_index' is your FAISS index that has been populated with document embeddings.
# 'document_map' is a dictionary that maps the index positions to your document contents (texts).
# 'query_embedding' is the embedding for the user query.

# Function to search FAISS index and retrieve the top k nearest documents
def retrieve_documents_from_query(query_embedding, k=3, index=index, model=model):  # Add model as argument
    """
    Retrieves documents from the FAISS index based on a query embedding.

    Args:
        query_embedding: The embedding of the query.
        k: The number of nearest documents to retrieve.
        index: The FAISS index.
        model: The model used to generate embeddings.

    Returns:
        A tuple containing the retrieved documents and their distances.
    """
    # Generate query embedding using the provided model
    query_embedding = model.encode([query_embedding])[0]  # Encode query and get the embedding

    # Convert query_embedding to a NumPy array with the correct data type and dimensions
    query_embedding = np.array([query_embedding]).astype(np.float32)

    # Get the dimensionality of the FAISS index
    index_dimension = index.d

    # Ensure the query embedding has the correct dimensionality
    if query_embedding.shape[1] != index_dimension:
        raise ValueError(f"Query embedding dimension ({query_embedding.shape[1]}) does not match index dimension ({index_dimension})")

    # Perform the search in the FAISS index
    distances, indices = index.search(query_embedding, k)  # Search top k results

    # Retrieve the corresponding documents using the indices
    retrieved_docs = [document_map[idx] for idx in indices[0]]

    return retrieved_docs, distances[0]


# Example query (use the actual query in your case)
query = "What is the process of material management in MES?"  # Use the query string directly

# Retrieve the top 3 most relevant documents based on the query
retrieved_documents, distances = retrieve_documents_from_query(query, k=3)  # Pass query to the function

# Print out the retrieved documents and their distances
for i, doc in enumerate(retrieved_documents):
    print(f"Result {i+1}:")
    print(f"Distance: {distances[i]}")
    print(f"Document Content: {doc}")
    print("\n")

Result 1:
Distance: 0.7817662954330444
Document Content: RAG_video


Result 2:
Distance: 1.9272916316986084
Document Content: RAG_text


Result 3:
Distance: 1.9272916316986084
Document Content: RAG_text_pdf




Example Code for Summarizing Based on Embeddings:



In [94]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Function to summarize documents using a summarization model
def summarize_documents(documents, max_length=200):
    # Load pre-trained T5 model and tokenizer for summarization
    model_name = "t5-small"  # You can use a larger model like "t5-base" or "t5-large" for better results
    tokenizer = T5Tokenizer.from_pretrained(model_name)
    model = T5ForConditionalGeneration.from_pretrained(model_name)

    # Concatenate the top documents to create a single input text
    combined_text = " ".join(documents)  # Ensure you are concatenating full content here, not just titles

    # Tokenize the input text
    inputs = tokenizer.encode("summarize: " + combined_text, return_tensors="pt", max_length=512, truncation=True)

    # Generate summary
    summary_ids = model.generate(inputs, max_length=max_length, num_beams=4, early_stopping=True)

    # Decode and return the summary
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# Example documents stored in your system (replace these with actual full-text content)
documents = [
    "Manufacturing Execution Systems (MES) are used to monitor and control manufacturing processes. An MES helps to optimize production scheduling, track inventory, and provide real-time updates on manufacturing operations.",
    "The Manufacturing Execution System is a critical component of modern manufacturing plants, facilitating the connection between raw material inputs and finished products through comprehensive data collection and process control.",
    "MES is a solution that supports businesses in streamlining production workflows, ensuring compliance with regulations, and improving quality control across manufacturing units."
]

# Summarize the documents
summary = summarize_documents(documents)
print("Generated Summary based on query embeddings:")
print(summary)


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Generated Summary based on query embeddings:
Manufacturing Execution Systems (MES) monitor and control manufacturing processes. an MES helps to optimize production scheduling, track inventory, and provide real-time updates on manufacturing operations.
